In [0]:
import pandas as pd
import pickle
import numpy as np
import time
st = time.time()
with open('/home/bil/huangtao/project/data/competition/data_new/share_visit.pickle') as f:
    share_visit_filter_10000=pickle.load(f)
print time.time() - st
print len(share_visit_filter_10000)
share_visit = share_visit_filter_10000
import time
st = time.time()
with open('/home/bil/huangtao/project/data/competition/data_new/negtive.pickle') as f:
    negtive=pickle.load(f)
print time.time() - st
print len(negtive)
share_list = [x.split('#') + [share_visit[x]] for x in share_visit]
data = pd.DataFrame(share_list,columns = ['bid1','bid2','share_num'])
data['label'] = 1
negtive = [x.split('#') for x in negtive]
data1= pd.DataFrame(negtive,columns = ['bid1','bid2'])
data1['share_num'] = 0
data1['label'] = 0
data = pd.concat((data,data1))

SyntaxError: ignored

In [0]:
data = data.reset_index(drop=True)
print data.shape
data.tail()
data_cut = data

(39262, 4)


In [0]:
from shapely.geometry import Point

In [0]:
import numpy as np
from shapely.geometry import Point
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
def get_dis(bid1,bid2):
    p1 = poi_dict[bid1]['loc']
    p2 = poi_dict[bid2]['loc']
    pp1 = [float(x) for x in p1]
    pp2 = [float(x) for x in p2]
    point1 = Point(pp1)
    point2 = Point(pp2)
    l = point1.distance(point2)
    return l
data_cut['dis'] = data_cut.apply(lambda x: get_dis(x['bid1'],x['bid2']),axis = 1)
data_cut['bid1_tag1'] = data_cut.apply(lambda x: poi_dict[x['bid1']]['std_tag'].split(';')[0],axis = 1)
data_cut['bid2_tag1'] = data_cut.apply(lambda x: poi_dict[x['bid2']]['std_tag'].split(';')[0],axis = 1)
data_cut['bid1_tag2'] = data_cut.apply(lambda x: poi_dict[x['bid1']]['std_tag'].split(';')[-1],axis = 1)
data_cut['bid2_tag2'] = data_cut.apply(lambda x: poi_dict[x['bid2']]['std_tag'].split(';')[-1],axis = 1)
data_cut = data_cut[data_cut['bid1_tag2'] != '']
data_cut = data_cut[data_cut['bid2_tag2'] != '']
data_cut = data_cut.reset_index(drop=True)
data_cut['bid1_loc1'] = data_cut.apply(lambda x: float(poi_dict[x['bid1']]['loc'][0]),axis = 1)
data_cut['bid1_loc2'] = data_cut.apply(lambda x: float(poi_dict[x['bid1']]['loc'][1]),axis = 1)
data_cut['bid2_loc1'] = data_cut.apply(lambda x: float(poi_dict[x['bid2']]['loc'][0]),axis = 1)
data_cut['bid2_loc2'] = data_cut.apply(lambda x: float(poi_dict[x['bid2']]['loc'][1]),axis = 1)
print data_cut.shape
data_cut.head()

(39262, 13)


,bid1,bid2,share_num,label,dis,bid1_tag1,bid2_tag1,bid1_tag2,bid2_tag2,bid1_loc1,bid1_loc2,bid2_loc1,bid2_loc2
0,1261398583068852223,15754384686349246450,14,1,1596.932695,美食,美食,中餐厅,中餐厅,12969494.76,4808920.76,12970784.18,4809862.88
1,1266464690268012543,1266561245427793919,5,1,1418.848770,购物,购物,超市,超市,12948343.92,4817465.31,12949485.42,4816622.63
2,1260297628332064767,1266418708348141567,12,1,1299.227377,酒店,酒店,快捷酒店,快捷酒店,12953362.51,4844432.46,12952276.78,4843718.89
3,1266631584107200511,10262250638989006701,5,1,9799.870485,美食,美食,小吃快餐店,小吃快餐店,12987107.03,4857249.03,12979917.52,4850589.57
4,15612544757557181497,17739908138645583145,13,1,3329.349735,购物,购物,超市,超市,12948917.79,4803358.46,12951696.58,4801524.64


In [0]:
def poi_format(cons):
    bid = cons[0]
    name = cons[2]
    std_tag = cons[11]
    aoi_id = cons[13]
    aoi_name = cons[14]
    point_x = cons[19]
    point_y = cons[20]
    parent_id = cons[31]
    return {'bid':bid, 'name':name, 'std_tag':std_tag, 
            'loc':(point_x,point_y), 'parent_id':parent_id,
           'aoi_id':aoi_id,' aoi_name':aoi_name}
import time
poi_file = '/home/bil/huangtao/project/data/poi_basic/poi_131/poi_131'
poi_dict = {}
flag = 1
st = time.time()
with open(poi_file) as f:
    for line in f:
        cons = eval(line)
        bid = cons[0]
        one_dict = poi_format(cons)
        poi_dict[bid] = one_dict
        if flag % 10000 == 0:
            t = time.time() - st
            print '%d / %d time:%.2f s\r' %(flag,1204309,t*(1204309-flag)/flag),
        flag += 1

# 构建图

In [0]:
import pickle
with open('/home/bil/huangtao/project/data/competition/data_new/co_map_filter.pickle','rb') as f:
    co_map_filter = pickle.load(f)#{town_id:[bids]}

In [0]:
import os
import time
import pandas as pd
import numpy as np
import itertools

class Edge(object):
    def __init__(self, start, end, label):
        self.label = str(label)
        self.start = start
        self.end = end
        self.str = self.__str__()

    def __str__(self):
        return self.label 


class Node(object):
    def __init__(self, name):
        self.name = name
        self.neighbors = set()
        self.neighbor2edges = {} # a dict of the form {node: [edge1, edge2, ...]}
        self.neighbor2edgesstr = {} # a dict of the form {node: [edge1.str, edge2.str, ...]}
        
    def __str__(self):
        return "Node({})".format(self.name)

    def add_edge(self, edge):
        self.neighbors.add(edge.end)
        self.neighbor2edges[edge.end] = self.neighbor2edges.get(edge.end, []) + [edge]
        self.neighbor2edgesstr[edge.end] = self.neighbor2edgesstr.get(edge.end, []) + [edge.str]
        
class Graph(object):
    def __init__(self):
        self.nodes = {} # nodes are indexed by name

    def get_node(self, name, create=False):
        if create and not name in self.nodes:
            self.nodes[name] = Node(name)
        return self.nodes[name]

    def partial_build_from_df(self, df):
        """Arguments:
        - `df` (pandas.DataFrame): DataFrame with three columns: head, tail and relation.
        Each row represents a triple in a knowledge graph.
        """
        flag = 0
        l = len(df)
        for idx,row in df.iterrows():
            flag += 1
            head = self.get_node(row['head'], create=True)
            tail = self.get_node(row['tail'], create=True)
            relation = row['relation']
            head.add_edge(Edge(head, tail, relation))
            tail.add_edge(Edge(tail, head, relation))
            if flag % 1000 == 0:
                print '%d / %d\r' %(flag, l),
    def partial_build_from_list(self, data_list):
        """Arguments:
        - `df` (pandas.DataFrame): DataFrame with three columns: head, tail and relation.
        Each row represents a triple in a knowledge graph.
        """
        flag = 0
        l = len(data_list)
        for pair in data_list:
            flag += 1
            bid1 = line[0]
            val = line[1]
            bid2 = line[2]
            head = self.get_node(bid1, create=True)
            tail = self.get_node(bid2, create=True)
            relation = val
            head.add_edge(Edge(head, tail, relation))
            tail.add_edge(Edge(tail, head, relation))
            if flag % 1000 == 0:
                print '%d / %d\r' %(flag, l),

In [0]:
#---------------co_map的图----------
graph = Graph()
flag = 0
l = len(co_map_filter)
for pair in co_map_filter:
    flag += 1
    bid1 = pair.split('#')[0]
    bid2 = pair.split('#')[1]
    val = co_map_filter[pair].strip('\n')
    head = graph.get_node(bid1, create=True)
    tail = graph.get_node(bid2, create=True)
    relation = val
    head.add_edge(Edge(head, tail, relation))
    tail.add_edge(Edge(tail, head, relation))
    if flag % 1000 == 0:
        print '%d / %d\r' %(flag, l),

In [0]:
len(co_map_filter)

3902181

In [0]:
'''
all_index = range(len(data))
from sklearn.model_selection import train_test_split
train_index,test_index = train_test_split(all_index,test_size=0.1,random_state=2)'''

In [0]:
from sklearn import metrics

In [0]:
def change_score(score, label):
    max_score = max(score)
    predict = [float(x)/max_score for x in score]
    p = predict
    predict = [int(x + 0.5) for x in predict]
    acc = metrics.accuracy_score(predict,label)
    max_acc = 0
    max_score = max(score)
    predict = [float(x)/max_score for x in score]
    res = []
    for i in range(20,80):
        j = float(i)/100
        t = [int(x + j) for x in p]
        acc1 = metrics.accuracy_score(t,label)
        if acc1 > max_acc:
            max_acc = acc1
            pp = t
    print 'acc:',acc,'max_acc:',max_acc
    return p,pp

In [0]:
def get_score(t, p, pp):
    acc = metrics.accuracy_score(t,pp)
    f1 = metrics.f1_score(t,pp)
    auc = metrics.roc_auc_score(t,p)
    precision = metrics.precision_score(t,pp)
    recall = metrics.recall_score(t,pp)
    print 'acc: ',acc,'auc: ',auc,'f1_score: ',f1,'\nprecision: ',precision, 'recall: ', recall
    return

# model1:co-map_val

In [0]:
score = []
for i in range(len(data)):
    res = []
    bid1 = data.loc[i,'bid1']
    bid2 = data.loc[i,'bid2']
    pair = bid1 + '#' + bid2
    pair1 = bid2 + '#' + bid1
    if pair in co_map_filter:
        val = int(co_map_filter[pair].strip('\n'))
    elif pair1 in co_map_filter:
        val = int(co_map_filter[pair1].strip('\n'))
    else:
        val = 0
    score += [val]

In [0]:
p,pp = change_score(score, label)

acc: 0.4574907034791911 max_acc: 0.4617441801232744


In [0]:
l = [1-x for x in label]
p,pp = change_score(score, l)

acc: 0.5425092965208089 max_acc: 0.5432224542814935


In [0]:
get_score(l,p,pp)

acc:  0.5432224542814935 auc:  0.3536510722018138 f1_score:  0.0 
precision:  0.0 recall:  0.0


# PA

In [0]:
score = []
for i in range(len(data)):
    val = 1.0
    bid1 = data.loc[i,'bid1']
    bid2 = data.loc[i,'bid2']
    try:
        head = graph.nodes[bid1]
        head_nears = head.neighbor2edgesstr
        for head_near in head_nears:
            val *= float(head_nears[head_near][0]) /10000.0
    except:
        val = 1.0
    try:
        tail = graph.nodes[bid2]
        tail_nears = tail.neighbor2edgesstr
        for tail_near in tail_nears:
            val *= float(tail_nears[tail_near][0]) / 10000.0
    except:
        pass
    if bid1 not in graph.nodes and bid2 not in graph.nodes:
        val = 0.0
    score += [val]
    if i % 100 == 0:print '\r%d / %d' %(i, len(data)),

39200 / 39262                                                                                  


In [0]:
p,pp = change_score(score, label)
l = [1-x for x in label]
p,pp = change_score(score, l)
get_score(l,p,pp)

acc: 0.45672660587845754 max_acc: 0.4570067749987265
acc: 0.5432733941215424 max_acc: 0.5432988640415669
acc:  0.5432988640415669 auc:  0.5603408626368782 f1_score:  0.00011152623654714773 
precision:  1.0 recall:  5.5766227972339954e-05


# CN

In [0]:
score = []
for i in range(len(data)):
    val = 0
    bid1 = data.loc[i,'bid1']
    bid2 = data.loc[i,'bid2']
    try:
        head = graph.nodes[bid1]
        tail = graph.nodes[bid2]
    except:
        val = 0
        score += [val]
        continue
    head_nears = tail.neighbor2edgesstr
    tail_nears = tail.neighbor2edgesstr
    for head_near in head_nears:
        if head_near in tail_nears:
            val += int(head_nears[head_near][0])
    for tail_near in tail_nears:
        if tail_near in head_nears:
            val += int(tail_nears[tail_near][0])
    score += [val]
    if i % 100 == 0:print '\r%d / %d' %(i, len(data)),

39200 / 39262                                                                                          


In [0]:
p,pp = change_score(score, label)
l = [1-x for x in label]
p,pp = change_score(score, l)
get_score(l,p,pp)

acc: 0.4594773572410983 max_acc: 0.47183026845295706
acc: 0.5405226427589017 max_acc: 0.5436299730018848
acc:  0.5436299730018848 auc:  0.4086741112620172 f1_score:  0.009508015478164732 
precision:  0.5443037974683544 recall:  0.004795895605621236


# RA

In [0]:
score = []
for i in range(300):
    val = 0
    bid1 = data.loc[i,'bid1']
    bid2 = data.loc[i,'bid2']
    try:
        head = graph.nodes[bid1]
        tail = graph.nodes[bid2]
    except:
        val = 0
        score += [val]
        continue
    head_nears = tail.neighbor2edgesstr
    tail_nears = tail.neighbor2edgesstr
    for head_near in head_nears:
        if head_near in tail_nears:
            near_sum = int(head_nears[head_near][0]) + int(tail_nears[head_near][0])
            ex_sum = 0
            head_near_nears = head_near.neighbor2edgesstr
            for head_near_near in head_near_nears:
                ex_sum += int(head_near_nears[head_near_near][0])
            val += float(near_sum) / ex_sum
    score += [val]
    if i % 1 == 0:print '\r%d / %d' %(i, len(data)),

for i in range(len(data)-300,len(data)):
    val = 0
    bid1 = data.loc[i,'bid1']
    bid2 = data.loc[i,'bid2']
    try:
        head = graph.nodes[bid1]
        tail = graph.nodes[bid2]
    except:
        val = 0
        score += [val]
        continue
    head_nears = tail.neighbor2edgesstr
    tail_nears = tail.neighbor2edgesstr
    for head_near in head_nears:
        if head_near in tail_nears:
            near_sum = int(head_nears[head_near][0]) + int(tail_nears[head_near][0])
            ex_sum = 0
            head_near_nears = head_near.neighbor2edgesstr
            for head_near_near in head_near_nears:
                ex_sum += int(head_near_nears[head_near_near][0])
            val += float(near_sum) / ex_sum
    score += [val]
    if i % 1 == 0:print '\r%d / %d' %(i, len(data)),

39261 / 39262                                                                                                                                                                                                                                                                                                                                       


In [0]:
ls = [1]*300 + [0]*300
p,pp = change_score(score, ls)
l = [1-x for x in ls]
p,pp = change_score(score, l)
get_score(l,p,pp)

acc: 0.5066666666666667 max_acc: 0.515
acc: 0.49333333333333335 max_acc: 0.5183333333333333
acc:  0.5183333333333333 auc:  0.5422833333333333 f1_score:  0.20821917808219179 
precision:  0.5846153846153846 recall:  0.12666666666666668


# Jaccard

In [0]:
score = []
for i in range(1000):
    val = 0
    bid1 = data.loc[i,'bid1']
    bid2 = data.loc[i,'bid2']
    try:
        head = graph.nodes[bid1]
        tail = graph.nodes[bid2]
    except:
        val = 0
        score += [val]
        continue
    head_nears = tail.neighbor2edgesstr
    tail_nears = tail.neighbor2edgesstr
    ex_sum = 0
    for head_near in head_nears:
        ex_sum += int(head_nears[head_near][0])
    for tail_near in tail_nears:
        ex_sum += int(tail_nears[tail_near][0])
        
    for head_near in head_nears:
        if head_near in tail_nears:
            near_sum = int(head_nears[head_near][0]) + int(tail_nears[head_near][0])
            val += float(near_sum) / ex_sum
    score += [val]
    if i % 1 == 0:print '\r%d / %d' %(i, len(data)),

for i in range(len(data)-1000,len(data)):
    val = 0
    bid1 = data.loc[i,'bid1']
    bid2 = data.loc[i,'bid2']
    try:
        head = graph.nodes[bid1]
        tail = graph.nodes[bid2]
    except:
        val = 0
        score += [val]
        continue
    head_nears = tail.neighbor2edgesstr
    tail_nears = tail.neighbor2edgesstr
    ex_sum = 0
    for head_near in head_nears:
        ex_sum += int(head_nears[head_near][0])
    for tail_near in tail_nears:
        ex_sum += int(tail_nears[tail_near][0])
        
    for head_near in head_nears:
        if head_near in tail_nears:
            near_sum = int(head_nears[head_near][0]) + int(tail_nears[head_near][0])
            val += float(near_sum) / ex_sum
    score += [val]
    if i % 1 == 0:print '\r%d / %d' %(i, len(data)),

39261 / 39262            


In [0]:
ls = [1]*1000 + [0]*1000
p,pp = change_score(score, ls)
get_score(ls,p,pp)

acc: 0.5215 max_acc: 0.5215
acc:  0.5215 auc:  0.5397875 f1_score:  0.6716981132075471 
precision:  0.5112271540469974 recall:  0.979


#              word2vec

In [0]:
import sys
import argparse
import numpy as np
import networkx as nx
from gensim.models import Word2Vec

In [0]:
val_list = []
for pair in co_map_filter:
    bid1 = pair.split('#')[0]
    bid2 = pair.split('#')[1]
    val = co_map_filter[pair].strip('\n')
    val_list += [int(val)]

In [0]:
float(sum(val_list)) / len(val_list)

182.82997047036002

In [0]:
co_map_nodevec = {}
val_list = []
for pair in co_map_filter:
    bid1 = pair.split('#')[0]
    bid2 = pair.split('#')[1]
    val = co_map_filter[pair].strip('\n')
    if int(val) > 182:
        co_map_nodevec[pair] = co_map_filter[pair]

In [0]:
print len(co_map_nodevec), len(co_map_filter)

702672 3902181


In [0]:
#---------------co_map的图----------
G = nx.Graph()
flag = 0
l = len(co_map_nodevec)
for pair in co_map_nodevec:
    flag += 1
    bid1 = pair.split('#')[0]
    bid2 = pair.split('#')[1]
    val = co_map_filter[pair].strip('\n')
    tag = poi_dict[bid1]['std_tag'].split(';')[0]
    G.add_node(int(bid1))  
    G.add_node(int(bid2))
    G.add_edge(int(bid1),int(bid2),weight=float(val))
    if flag % 1000 == 0:
        print '%d / %d\r' %(flag, l),

In [0]:
import numpy as np
import networkx as nx
import random


class node2vecgraph():
    def __init__(self, nx_G, is_directed, p, q):
        self.G = nx_G
        self.is_directed = is_directed
        self.p = p
        self.q = q

    def node2vec_walk(self, walk_length, start_node):
        '''
        Simulate a random walk starting from start node.
        '''
        G = self.G
        alias_nodes = self.alias_nodes
        alias_edges = self.alias_edges

        walk = [start_node]

        while len(walk) < walk_length:
            cur = walk[-1]
            cur_nbrs = sorted(G.neighbors(cur))
            if len(cur_nbrs) > 0:
                if len(walk) == 1:
                    walk.append(cur_nbrs[alias_draw(alias_nodes[cur][0], alias_nodes[cur][1])])
                else:
                    prev = walk[-2]
                    next = cur_nbrs[alias_draw(alias_edges[(prev, cur)][0], 
                        alias_edges[(prev, cur)][1])]
                    walk.append(next)
            else:
                break

        return walk

    def simulate_walks(self, num_walks, walk_length):
        '''
        Repeatedly simulate random walks from each node.
        '''
        G = self.G
        walks = []
        nodes = list(G.nodes())
        print 'Walk iteration:'
        for walk_iter in range(num_walks):
            print str(walk_iter+1), '/', str(num_walks)
            random.shuffle(nodes)
            for node in nodes:
                walks.append(self.node2vec_walk(walk_length=walk_length, start_node=node))

        return walks

    def get_alias_edge(self, src, dst):
        '''
        Get the alias edge setup lists for a given edge.
        '''
        G = self.G
        unnormalized_probs = []
        for dst_nbr in sorted(G.neighbors(dst)):
            unnormalized_probs.append(G[dst][dst_nbr]['weight'])
        norm_const = sum(unnormalized_probs)
        normalized_probs =  [float(u_prob)/norm_const for u_prob in unnormalized_probs]

        return alias_setup(normalized_probs)

    def preprocess_transition_probs(self):
        '''
        Preprocessing of transition probabilities for guiding the random walks.
        '''
        G = self.G
        is_directed = self.is_directed

        alias_nodes = {}
        flag = 0
        l = G.number_of_nodes()
        print 'building...'
        for node in G.nodes():
            flag+=1
            unnormalized_probs = [G[node][nbr]['weight'] for nbr in sorted(G.neighbors(node))]
            norm_const = sum(unnormalized_probs)
            normalized_probs =  [float(u_prob)/norm_const for u_prob in unnormalized_probs]
            alias_nodes[node] = alias_setup(normalized_probs)
            if flag % 10 == 0:
                print '\r%d / %d' %(flag, l),

        alias_edges = {}
        triads = {}
        print '\nworking...'
        if is_directed:
            for edge in G.edges():
                alias_edges[edge] = self.get_alias_edge(edge[0], edge[1])
        else:
            l = G.number_of_edges()
            flag = 0
            for edge in G.edges():
                flag+=1
                alias_edges[edge] = self.get_alias_edge(edge[0], edge[1])
                alias_edges[(edge[1], edge[0])] = self.get_alias_edge(edge[1], edge[0])
                if flag % 100 == 0:
                    print '\r%d / %d' %(flag, l),
        self.alias_nodes = alias_nodes
        self.alias_edges = alias_edges

        return


def alias_setup(probs):
    '''
    Compute utility lists for non-uniform sampling from discrete distributions.
    Refer to https://hips.seas.harvard.edu/blog/2013/03/03/the-alias-method-efficient-sampling-with-many-discrete-outcomes/
    for details
    '''
    K = len(probs)
    q = np.zeros(K)
    J = np.zeros(K, dtype=np.int)

    smaller = []
    larger = []
    for kk, prob in enumerate(probs):
        q[kk] = K*prob
        if q[kk] < 1.0:
            smaller.append(kk)
        else:
            larger.append(kk)

    while len(smaller) > 0 and len(larger) > 0:
        small = smaller.pop()
        large = larger.pop()

        J[small] = large
        q[large] = q[large] + q[small] - 1.0
        if q[large] < 1.0:
            smaller.append(large)
        else:
            larger.append(large)

    return J, q

def alias_draw(J, q):
    '''
    Draw sample from a non-uniform discrete distribution using alias sampling.
    '''
    K = len(J)

    kk = int(np.floor(np.random.rand()*K))
    if np.random.rand() < q[kk]:
        return kk
    else:
        return J[kk]

In [0]:
Gn = node2vecgraph(G, False, 1, 1)
Gn.preprocess_transition_probs()

building...
129400 / 129403                              
working...
702600 / 702672                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [0]:
walks = Gn.simulate_walks(10, 10)

Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10


In [0]:
def learn_embeddings(walks):
	'''
	Learn embeddings by optimizing the Skipgram objective using SGD.
	'''
	walks = [map(str, walk) for walk in walks]
	model = Word2Vec(walks, size=128, window=10, min_count=0, sg=1, workers=8, iter=1)

	return model

In [0]:
def cosine_similarity(vector1, vector2):
    dot_product = 0.0
    normA = 0.0
    normB = 0.0
    for a, b in zip(vector1, vector2):
        dot_product += a * b
        normA += a ** 2
        normB += b ** 2
    if normA == 0.0 or normB == 0.0:
        return 0
    else:
        return round(dot_product / ((normA**0.5)*(normB**0.5)) * 100, 2)

In [0]:
vector = learn_embeddings(walks)

In [0]:
t = []
p = []
f = []
for i in range(len(data)):
    val = 0
    bid1 = data.loc[i,'bid1']
    bid2 = data.loc[i,'bid2']
    tag1 = poi_dict[bid1]['std_tag'].split(';')[0]
    try:
        m1 = vector[bid1]
    except:
        m1 = [0.0] * 128
    try:
        m2 = vector[bid2]
    except:
        m2 = [0.0] * 128
    cos = cosine_similarity(m1, m2)
    f += [list(m1) + list(m2)]
    p += [cos]
    t += [data.loc[i,'label']]    

/home/bil/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.
/home/bil/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [0]:
p,pp = change_score(p, label)
get_score(label,p,pp)

acc: 0.5813254546380724 max_acc: 0.613086444908563
acc:  0.613086444908563 auc:  0.6916650457073914 f1_score:  0.686414961914002 
precision:  0.6132113746173422 recall:  0.7794655414908579


In [0]:
train_x = np.array(f)[train_index,:]
train_y = np.array(t)[train_index]
test_x = np.array(f)[test_index,:]
test_y = np.array(t)[test_index]

In [0]:
print np.array(f).shape

(39262, 256)


In [0]:
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
print '逻辑回归模型:\n',model

逻辑回归模型:
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


In [0]:
model.fit(train_x,train_y)
score = model.predict_proba(test_x)  #预测分类概率
predict_y = model.predict(test_x)
acc = model.score(test_x,test_y)
print acc

0.727782021899669


In [0]:
pp = [int(x[1]+0.5) for x in score]
get_score(test_y,score[:,1],pp)

acc:  0.727782021899669 auc:  0.7972701089560634 f1_score:  0.7474604299551146 
precision:  0.7371854613233924 recall:  0.7580258744609487


# feature-MLP

In [0]:
data_cut = data
from sklearn import preprocessing
dis_scaled = preprocessing.scale(data_cut.loc[:,'dis'])
co_map_list = []
for i in range(len(data_cut)):
    flag +=1
    res = []
    bid1 = data_cut.loc[i,'bid1']
    bid2 = data_cut.loc[i,'bid2']
    pair = bid1 + '#' + bid2
    pair1 = bid2 + '#' + bid1
    if pair in co_map_filter:
        val = int(co_map_filter[pair].strip('\n'))
    elif pair1 in co_map_filter:
        val = int(co_map_filter[pair1].strip('\n'))
    else:
        val = 0
    co_map_list += [val]
co_map_scaled = preprocessing.scale(co_map_list)
co_feature = np.concatenate([np.array([dis_scaled]).T, 
                         np.array([co_map_scaled]).T],axis = 1)
print co_feature.shape

(39262, 2)


In [0]:
with open('/home/bil/huangtao/project/data/competition/region_data/region_feature_dict_with_profile.pickle','rb') as f:
    region_feature = pickle.load(f)  
len(region_feature)

320

In [0]:
import pickle
with open('/home/bil/huangtao/project/data/competition/data_new/bid_town_dict.pickle','rb') as f:
    bid_town_dict = pickle.load(f)
import pickle
with open('/home/bil/huangtao/project/data/competition/region_data/region_feature_dict_with_profile.pickle','rb') as f:
    region_feature = pickle.load(f)  
bid1_region = []
for bid in data_cut['bid1']:
    if bid not in bid_town_dict:
        bid1_region +=[dict()]
        continue
    region_ids = bid_town_dict[bid]
    region_id = region_ids[0]
    feature = region_feature[region_id]
    if len(region_ids) != 1:
        region_id = region_ids[1]
        feature1 = region_feature[region_id]
        keys = list(set(feature.keys() + feature1.keys()))
        for key in keys:
            if key in feature and key in feature1:
                feature[key] = (float(feature[key]) + float(feature1[key]))/2.0
            elif key in feature:
                feature[key] = feature[key]/2.0
            elif key in feature1:    
                feature[key] = feature1[key]/2.0
    bid1_region += [feature]
bid2_region = []
for bid in data_cut['bid2']:
    if bid not in bid_town_dict:
        bid2_region +=[dict()]
        continue
    region_ids = bid_town_dict[bid]
    region_id = region_ids[0]
    feature = region_feature[region_id]
    if len(region_ids) != 1:
        region_id = region_ids[1]
        feature1 = region_feature[region_id]
        keys = list(set(feature.keys() + feature1.keys()))
        for key in keys:
            if key in feature and key in feature1:
                feature[key] = (float(feature[key]) + float(feature1[key]))/2.0
            elif key in feature:
                feature[key] = feature[key]/2.0
            elif key in feature1:    
                feature[key] = feature1[key]/2.0
    bid2_region += [feature]
region1 = pd.DataFrame(bid1_region).fillna(0.0)
region2 = pd.DataFrame(bid2_region).fillna(0.0)
region2.head()

/home/bil/anaconda2/lib/python2.7/site-packages/pandas/core/indexes/base.py:1592: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  return key in self._engine


,丽人_visit,交通设施_hot,交通设施_num,亲子教育_visit,便利店_num,健身中心_hot,其他_hot,典当行_num,内部楼栋_hot,动物园_num,...,地铁、轻轨_visit,省道_num,省道_hot,省道_visit,飞机场_num,飞机场_hot,飞机场_visit,城市次干道_num,城市次干道_hot,城市次干道_visit
0,4092.000000,9183.00000,563.000000,7.300000e+01,45.000000,989.000000,14102.000000,5.000000,8869.000000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,14301.000000,55700.00000,2556.000000,8.930000e+02,218.000000,2216.000000,61127.000000,18.000000,32267.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,804.993729,661.59137,216.930733,2.618645e-57,4.174034,331.323006,4205.153839,0.825966,985.030281,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1268.000000,27535.00000,1962.000000,0.000000e+00,21.000000,122.000000,31633.000000,0.000000,838.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8913.000000,13616.00000,736.000000,6.040000e+02,114.000000,1442.000000,22621.000000,8.000000,13761.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
region1_scaled = preprocessing.scale(region1)
region2_scaled = preprocessing.scale(region2)

In [0]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
def labels_to_onehot(label):
    enc_label = LabelEncoder()
    enc_label.fit(label)
    encode = enc_label.transform(label)
    encode = np.array([encode]).T
    enc_onehot = OneHotEncoder()
    enc_onehot.fit(encode)
    onehot = enc_onehot.transform(encode)
    return onehot
all_bid = set()
for pair in co_map_filter:
    bid1 = pair.split('#')[0]
    bid2 = pair.split('#')[1]
    all_bid.add(bid1)
    all_bid.add(bid2)
for bid in data_cut['bid1']:
    all_bid.add(bid)
for bid in data_cut['bid2']:
    all_bid.add(bid)   
bid_list = list(all_bid)
tag1_list = []
tag2_list = []
for bid in bid_list:
    tag1 = poi_dict[bid]['std_tag'].split(';')[0]
    tag2 = poi_dict[bid]['std_tag'].split(';')[-1]
    tag1_list += [tag1]
    tag2_list += [tag2]
onehot1 = labels_to_onehot(tag1_list)
onehot2 = labels_to_onehot(tag2_list)
loc1_list = []
loc2_list = []
for bid in bid_list:
    loc1 = poi_dict[bid]['loc'][0]
    loc2 = poi_dict[bid]['loc'][1]
    loc1_list += [loc1]
    loc2_list += [loc2]
loc1_scaled = preprocessing.scale(loc1_list)
loc2_scaled = preprocessing.scale(loc2_list)  

In [0]:
all_bid = set()
for pair in co_map_filter:
    bid1 = pair.split('#')[0]
    bid2 = pair.split('#')[1]
    all_bid.add(bid1)
    all_bid.add(bid2)
for bid in data_cut['bid1']:
    all_bid.add(bid)
for bid in data_cut['bid2']:
    all_bid.add(bid)  
all_bid = list(all_bid)
i = 0
poi_index = {}
for bid in all_bid:
    i+=1
    poi_index[bid] = i

In [0]:
poi_embed_dict = {}
flag = 0
for i in range(len(bid_list)):
    flag += 1
    bid = bid_list[i]
    tag1 = onehot1[i]
    tag2 = onehot2[i]
    loc1 = loc1_scaled[i]
    loc2 = loc2_scaled[i]
    poi_embed_dict[bid] = np.array([tag1.toarray()[0].tolist() + tag2.toarray()[0].tolist() + [loc1]+[loc2]])
    if flag % 1000 == 0:print '%d / %d\r' %(flag, len(bid_list)),

In [0]:
bid1_embed = []
for bid in data_cut['bid1']:
    embed = poi_embed_dict[bid][0]
    bid1_embed += [embed]
bid2_embed = []
for bid in data_cut['bid2']:
    embed = poi_embed_dict[bid][0]
    bid2_embed += [embed]
bid1_embed = np.array(bid1_embed)
bid2_embed = np.array(bid2_embed)
print bid1_embed.shape, bid2_embed.shape

(39262, 211) (39262, 211)


In [0]:
len(co_map_filter)

3902181

In [0]:
label = np.array(data_cut.loc[:,'label'])
all_index = range(len(data))
from sklearn.model_selection import train_test_split
train_index,test_index = train_test_split(all_index,test_size=0.1,random_state=2)

In [0]:
import pickle
with open('/home/bil/huangtao/project/data/competition/data_new/gpu_motif/co_feature.pickle','wb') as f:
    pickle.dump(co_feature,f)
with open('/home/bil/huangtao/project/data/competition/data_new/gpu_motif/bid1_embed.pickle','wb') as f:
    pickle.dump(bid1_embed,f)
with open('/home/bil/huangtao/project/data/competition/data_new/gpu_motif/bid2_embed.pickle','wb') as f:
    pickle.dump(bid2_embed,f) 
with open('/home/bil/huangtao/project/data/competition/data_new/gpu_motif/region1_embed.pickle','wb') as f:
    pickle.dump(region1_scaled,f) 
with open('/home/bil/huangtao/project/data/competition/data_new/gpu_motif/region2_embed.pickle','wb') as f:
    pickle.dump(region2_scaled,f)
with open('/home/bil/huangtao/project/data/competition/data_new/gpu_motif/train_index.pickle','wb') as f:
    pickle.dump(train_index,f)
with open('/home/bil/huangtao/project/data/competition/data_new/gpu_motif/test_index.pickle','wb') as f:
    pickle.dump(test_index,f)
with open('/home/bil/huangtao/project/data/competition/data_new/gpu_motif/label.pickle','wb') as f:
    pickle.dump(np.array(data_cut.loc[:,'label']),f)

In [0]:
import pickle
with open('/home/bil/huangtao/project/data/competition/data_new/gpu_motif/co_feature.pickle','rb') as f:
    co_feature = pickle.load(f)
with open('/home/bil/huangtao/project/data/competition/data_new/gpu_motif/bid1_embed.pickle','rb') as f:
    bid1_embed = pickle.load(f)
with open('/home/bil/huangtao/project/data/competition/data_new/gpu_motif/bid2_embed.pickle','rb') as f:
    bid2_embed = pickle.load(f)
with open('/home/bil/huangtao/project/data/competition/data_new/gpu_motif/region1_embed.pickle','rb') as f:
    region1 = pickle.load(f)
with open('/home/bil/huangtao/project/data/competition/data_new/gpu_motif/region2_embed.pickle','rb') as f:
    region2 = pickle.load(f)

In [0]:
with open('/home/bil/huangtao/project/data/competition/data_new/gpu_motif/train_index.pickle','rb') as f:
    train_index = pickle.load(f)
with open('/home/bil/huangtao/project/data/competition/data_new/gpu_motif/test_index.pickle','rb') as f:
    test_index = pickle.load(f)
with open('/home/bil/huangtao/project/data/competition/data_new/gpu_motif/label.pickle','rb') as f:
    label = pickle.load(f)

In [0]:
data_x = np.concatenate([co_feature, bid1_embed, bid2_embed],axis = 1)

In [0]:
data_x = np.array(data_x)
print data_x.shape

(39262, 424)


In [0]:
train_y = label[train_index]
test_y = label[test_index]
train_x = data_x[train_index,:]
test_x = data_x[test_index,:]

In [0]:
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
print '逻辑回归模型:\n',model

逻辑回归模型:
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


In [0]:
model.fit(train_x,train_y)
score = model.predict_proba(test_x)  #预测分类概率
predict_y = model.predict(test_x)
acc = model.score(test_x,test_y)
print acc

0.7030812324929971


In [0]:
pp = [int(x[1]+0.5) for x in score]
get_score(test_y,score[:,1],pp)

acc:  0.7030812324929971 auc:  0.777510364367409 f1_score:  0.7207854406130267 
precision:  0.7204404021062709 recall:  0.7211308097747964


# node2vec+feature

In [0]:
t = []
p = []
f = []
for i in range(len(data)):
    val = 0
    bid1 = data.loc[i,'bid1']
    bid2 = data.loc[i,'bid2']
    tag1 = poi_dict[bid1]['std_tag'].split(';')[0]
    try:
        m1 = vector[bid1]
    except:
        m1 = [0.0] * 128
    try:
        m2 = vector[bid2]
    except:
        m2 = [0.0] * 128
    cos = cosine_similarity(m1, m2)
    f += [list(m1) + list(m2)]
    p += [cos]
    t += [data.loc[i,'label']]   

/home/bil/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.
/home/bil/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [0]:
data_x = np.concatenate([co_feature, bid1_embed, f],axis = 1)

In [0]:
np.array(data_x).shape

(39262, 469)

In [0]:
train_y = np.array(t)[train_index]
test_y = np.array(t)[test_index]
train_x = np.array(data_x)[train_index,:]
test_x = np.array(data_x)[test_index,:]

In [0]:
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
print '逻辑回归模型:\n',model

逻辑回归模型:
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


In [0]:
model.fit(train_x,train_y)
score = model.predict_proba(test_x)  #预测分类概率
predict_y = model.predict(test_x)
acc = model.score(test_x,test_y)
print acc

0.7998472116119175


In [0]:
pp = [int(x[1]+0.5) for x in score]
get_score(test_y,score[:,1],pp)

acc:  0.7998472116119175 auc:  0.8802449428136914 f1_score:  0.8111484863046613 
precision:  0.8134939759036145 recall:  0.8088164829899377


In [0]:
import xgboost as xgb
xgbmodel = xgb.XGBClassifier(n_estimators=100, 
                             silent=True, 
                             objective='binary:logistic',
                             max_depth=2, 
                             learning_rate=0.1)
xgbmodel.fit(train_x,train_y,early_stopping_rounds=10,eval_set=[(test_x,test_y)],eval_metric="error")

[0]	validation_0-error:0.314489
Will train until validation_0-error hasn't improved in 10 rounds.
[1]	validation_0-error:0.30303
[2]	validation_0-error:0.297937
[3]	validation_0-error:0.286478
[4]	validation_0-error:0.292844
[5]	validation_0-error:0.281385
[6]	validation_0-error:0.279348
[7]	validation_0-error:0.269926
[8]	validation_0-error:0.271963
[9]	validation_0-error:0.273237
[10]	validation_0-error:0.271709
[11]	validation_0-error:0.269926
[12]	validation_0-error:0.263815
[13]	validation_0-error:0.263815
[14]	validation_0-error:0.255921
[15]	validation_0-error:0.2493
[16]	validation_0-error:0.253374
[17]	validation_0-error:0.2493
[18]	validation_0-error:0.248281
[19]	validation_0-error:0.240896
[20]	validation_0-error:0.246244
[21]	validation_0-error:0.239114
[22]	validation_0-error:0.238859
[23]	validation_0-error:0.235039
[24]	validation_0-error:0.232748
[25]	validation_0-error:0.233766
[26]	validation_0-error:0.229692
[27]	validation_0-error:0.231729
[28]	validation_0-error:0

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=2, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [0]:
score = xgbmodel.predict_proba(test_x)  #预测分类概率
predict_y = xgbmodel.predict(test_x)
pp = [int(x[1]+0.5) for x in score]
get_score(test_y,score[:,1],pp)

acc:  0.8105423987776929 auc:  0.892174251578092 f1_score:  0.8205499276411 
precision:  0.8261291889266634 recall:  0.8150455198850024


/home/bil/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


# XGBOOST

In [0]:
data_x = np.concatenate([co_feature, bid1_embed, bid2_embed, region1,region2],axis = 1)

In [0]:
np.array(data_x).shape

(39262, 1590)

In [0]:
train_y = label[train_index]
test_y = label[test_index]
train_x = np.array(data_x)[train_index,:]
test_x = np.array(data_x)[test_index,:]

In [0]:
print train_y.shape
print train_x.shape

(35335,)
(35335, 1590)


In [0]:
import xgboost as xgb
xgbmodel = xgb.XGBClassifier(n_estimators=100, 
                             silent=True, 
                             objective='binary:logistic',
                             max_depth=2, 
                             learning_rate=0.1)

In [0]:
xgbmodel.fit(train_x,train_y,early_stopping_rounds=10,eval_set=[(test_x,test_y)],eval_metric="error")

[0]	validation_0-error:0.307614
Will train until validation_0-error hasn't improved in 10 rounds.
[1]	validation_0-error:0.307614
[2]	validation_0-error:0.307614
[3]	validation_0-error:0.307614
[4]	validation_0-error:0.307614
[5]	validation_0-error:0.297173
[6]	validation_0-error:0.297173
[7]	validation_0-error:0.297937
[8]	validation_0-error:0.294118
[9]	validation_0-error:0.295391
[10]	validation_0-error:0.295391
[11]	validation_0-error:0.294118
[12]	validation_0-error:0.294372
[13]	validation_0-error:0.289279
[14]	validation_0-error:0.290043
[15]	validation_0-error:0.285205
[16]	validation_0-error:0.284696
[17]	validation_0-error:0.284186
[18]	validation_0-error:0.282149
[19]	validation_0-error:0.280367
[20]	validation_0-error:0.277311
[21]	validation_0-error:0.275274
[22]	validation_0-error:0.272218
[23]	validation_0-error:0.271199
[24]	validation_0-error:0.272982
[25]	validation_0-error:0.271454
[26]	validation_0-error:0.270945
[27]	validation_0-error:0.26738
[28]	validation_0-err

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=2, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [0]:
score = xgbmodel.predict_proba(test_x)  #预测分类概率
predict_y = xgbmodel.predict(test_x)
pp = [int(x[1]+0.5) for x in score]
get_score(test_y,score[:,1],pp)

acc:  0.7542653425006366 auc:  0.8289542144955313 f1_score:  0.7568657092466616 
precision:  0.798087141339001 recall:  0.7196933397220892


/home/bil/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


# motif

In [0]:
print 'load motif1..'
with open('/home/bil/huangtao/project/data/competition/data_new/motif1_dict.pickle', 'rb') as f:
    motif1_dict = pickle.load(f)
with open('/home/bil/huangtao/project/data/competition/data_new/motif1_negtive_dict.pickle', 'rb') as f:
    motif1_negtive_dict = pickle.load(f)
print 'load motif2..'
with open('/home/bil/huangtao/project/data/competition/data_new/motif2_dict.pickle', 'rb') as f:
    motif2_dict = pickle.load(f)
with open('/home/bil/huangtao/project/data/competition/data_new/motif2_negtive_dict.pickle', 'rb') as f:
    motif2_negtive_dict = pickle.load(f)
print 'load motif3..'
with open('/home/bil/huangtao/project/data/competition/data_new/motif3_dict.pickle', 'rb') as f:
    motif3_dict = pickle.load(f)
with open('/home/bil/huangtao/project/data/competition/data_new/motif3_negtive_dict.pickle', 'rb') as f:
    motif3_negtive_dict = pickle.load(f)
print 'load motif4..'
with open('/home/bil/huangtao/project/data/competition/data_new/motif4_dict.pickle', 'rb') as f:
    motif4_dict = pickle.load(f)
with open('/home/bil/huangtao/project/data/competition/data_new/motif4_negtive_dict.pickle', 'rb') as f:
    motif4_negtive_dict = pickle.load(f)

load motif1..
load motif2..
load motif3..
load motif4..


In [0]:
import time
import pickle
st = time.time()
with open('/home/bil/huangtao/project/data/competition/data_new/share_visit.pickle') as f:
    share_visit=pickle.load(f)
print time.time() - st
print len(share_visit)
import time
st = time.time()
with open('/home/bil/huangtao/project/data/competition/data_new/negtive.pickle') as f:
    negtive=pickle.load(f)
print time.time() - st
print len(negtive)

0.145349979401
21330
0.088681936264
17932


In [0]:
#------------motif1---------------
motif1_feature = []#[poi*3,pre,co_sq, dis_sq]*5
flag = 0
for i in range(len(data_cut)):
    flag +=1
    bid1 = data_cut.loc[i,'bid1']
    bid2 = data_cut.loc[i,'bid2']
    pair = bid1 + '#' + bid2
    if pair in motif1_dict:
        val = motif1_dict[pair]
    else:
        val = motif1_negtive_dict[pair]#[poi*3,pre,co_sq, dis_sq]*5
    embed1 = poi_embed_dict[bid1][0]
    embed2 = poi_embed_dict[bid2][0]
    line = val
    motif1_feature += [line]
    if flag % 100 == 0:print '%d / %d\r' %(flag, len(data_cut)),

NameError: name 'poi_embed_dict' is not defined

In [0]:
#------------motif2---------------
motif2_feature = []#[poi*3,pre,co_sq, dis_sq]*5
flag = 0
for i in range(len(data_cut)):
    flag +=1
    bid1 = data_cut.loc[i,'bid1']
    bid2 = data_cut.loc[i,'bid2']
    pair = bid1 + '#' + bid2
    if pair in motif2_dict:
        val = motif2_dict[pair]
    else:
        val = motif2_negtive_dict[pair]#[poi*3,pre,co_sq, dis_sq]*5
    line = val
    motif2_feature += [line]
    if flag % 100 == 0:print '%d / %d\r' %(flag, len(data_cut)),

In [0]:
#------------motif3---------------
motif3_feature = []#[poi*3,pre,co_sq, dis_sq]*5
flag = 0
for i in range(len(data_cut)):
    flag +=1
    bid1 = data_cut.loc[i,'bid1']
    bid2 = data_cut.loc[i,'bid2']
    pair = bid1 + '#' + bid2
    if pair in motif3_dict:
        val = motif3_dict[pair]
    else:
        val = motif3_negtive_dict[pair]#[poi*3,pre,co_sq, dis_sq]*5
    line = val
    motif3_feature += [line]
    if flag % 100 == 0:print '%d / %d\r' %(flag, len(data_cut)),

In [0]:
#------------motif4---------------
motif4_feature = []#[poi*3,pre,co_sq, dis_sq]*5
flag = 0
for i in range(len(data_cut)):
    flag +=1
    bid1 = data_cut.loc[i,'bid1']
    bid2 = data_cut.loc[i,'bid2']
    pair = bid1 + '#' + bid2
    if pair in motif4_dict:
        val = motif4_dict[pair]
    else:
        val = motif4_negtive_dict[pair]#[poi*3,pre,co_sq, dis_sq]*5
    line = val
    motif4_feature += [line]
    if flag % 100 == 0:print '%d / %d\r' %(flag, len(data_cut)),

In [0]:
poi_embed_dict_filter = {}
print 'add 1...'
for val in motif1_feature:
    for v in val:
        motif = v[0]
        if motif == []:continue
        poi_embed_dict_filter[motif[0]] = poi_embed_dict[motif[0]][0]
        poi_embed_dict_filter[motif[1]] = poi_embed_dict[motif[1]][0]
        poi_embed_dict_filter[motif[2]] = poi_embed_dict[motif[2]][0]
print 'add 2...'
for val in motif2_feature:
    for v in val:
        motif = v[0]
        if motif == []:continue
        poi_embed_dict_filter[motif[0]] = poi_embed_dict[motif[0]][0]
        poi_embed_dict_filter[motif[1]] = poi_embed_dict[motif[1]][0]
        poi_embed_dict_filter[motif[2]] = poi_embed_dict[motif[2]][0]
print 'add 3...'
for val in motif3_feature:
    for v in val:
        motif = v[0]
        if motif == []:continue
        poi_embed_dict_filter[motif[0]] = poi_embed_dict[motif[0]][0]
        poi_embed_dict_filter[motif[1]] = poi_embed_dict[motif[1]][0]
        poi_embed_dict_filter[motif[2]] = poi_embed_dict[motif[2]][0]
print 'add 4...'
for val in motif4_feature:
    for v in val:
        motif = v[0]
        if motif == []:continue
        poi_embed_dict_filter[motif[0]] = poi_embed_dict[motif[0]][0]
        poi_embed_dict_filter[motif[1]] = poi_embed_dict[motif[1]][0]
        poi_embed_dict_filter[motif[2]] = poi_embed_dict[motif[2]][0]

In [0]:
print 'save...'
with open('/home/bil/huangtao/project/data/competition/data_new/gpu_motif/poi_embed_dict.pickle','wb') as f:
    pickle.dump(poi_embed_dict_filter, f)
print 'save...'
with open('/home/bil/huangtao/project/data/competition/data_new/gpu_motif/motif1_feature.pickle','wb') as f:
    pickle.dump(motif1_feature,f)
print 'save...'
with open('/home/bil/huangtao/project/data/competition/data_new/gpu_motif/motif2_feature.pickle','wb') as f:
    pickle.dump(motif2_feature,f)
print 'save...'
with open('/home/bil/huangtao/project/data/competition/data_new/gpu_motif/motif3_feature.pickle','wb') as f:
    pickle.dump(motif3_feature,f)
print 'save...'
with open('/home/bil/huangtao/project/data/competition/data_new/gpu_motif/motif4_feature.pickle','wb') as f:
    pickle.dump(motif4_feature,f)

save...
save...
save...
save...
save...


In [0]:

with open('/home/bil/huangtao/project/data/competition/data_new/gpu_motif/motif1_feature.pickle','rb') as f:
    motif1_feature = pickle.load(f)

with open('/home/bil/huangtao/project/data/competition/data_new/gpu_motif/motif2_feature.pickle','rb') as f:
    motif2_feature = pickle.load(f)

with open('/home/bil/huangtao/project/data/competition/data_new/gpu_motif/motif3_feature.pickle','rb') as f:
    motif3_feature = pickle.load(f)

with open('/home/bil/huangtao/project/data/competition/data_new/gpu_motif/motif4_feature.pickle','rb') as f:
    motif4_feature = pickle.load(f)

save...
save...
save...
save...


In [0]:
with open('/home/bil/huangtao/project/data/competition/data_new/gpu_motif/poi_embed_dict.pickle','rb') as f:
     poi_embed_dict = pickle.load(f)

In [0]:
def build_motif_feature(motif_feature):
    res = []
    flag = 0
    for val in motif_feature:
        #val:[[],[],[],[]]*5
        flag += 1
        pre = val[0][1]
        if val[0][0] != []:
            pre += val[0][2] + val[0][3]
        else:
            pre += [0]*6
        line = np.array(pre)
        res += [line]
        if flag == 1:print 'motif_feature: ',line.shape
        if flag % 100 == 0:print '%d / %d\r' %(flag, len(motif_feature)),
    r = np.array(res)
    return r

In [0]:
m1 = build_motif_feature(motif1_feature)
m2 = build_motif_feature(motif2_feature)
m3 = build_motif_feature(motif3_feature)
m4 = build_motif_feature(motif4_feature)

motif_feature:  (106,)
motif_feature:  (106,)
motif_feature:  (106,)
motif_feature:  (106,)


In [0]:
co_feature.shape, bid1_embed.shape, bid2_embed.shape, region1.shape, m1.shape

((39262, 2), (39262, 211), (39262, 211), (39262, 583), (39262, 106))

In [0]:
data_x = np.concatenate([co_feature,bid1_embed,bid2_embed,m3],axis = 1)

In [0]:
data_x = np.array(data_x)
print data_x.shape
train_y = label[train_index]
test_y = label[test_index]
train_x = data_x[train_index,:]
test_x = data_x[test_index,:]

(39262, 530)


In [0]:
xgbmodel.fit(train_x,train_y,early_stopping_rounds=10,eval_set=[(test_x,test_y)],eval_metric="error")

[0]	validation_0-error:0.307614
Will train until validation_0-error hasn't improved in 10 rounds.
[1]	validation_0-error:0.307614
[2]	validation_0-error:0.307614
[3]	validation_0-error:0.307614
[4]	validation_0-error:0.307614
[5]	validation_0-error:0.307614
[6]	validation_0-error:0.299465
[7]	validation_0-error:0.298956
[8]	validation_0-error:0.294627
[9]	validation_0-error:0.299465
[10]	validation_0-error:0.29259
[11]	validation_0-error:0.289534
[12]	validation_0-error:0.289534
[13]	validation_0-error:0.289534
[14]	validation_0-error:0.289279
[15]	validation_0-error:0.282404
[16]	validation_0-error:0.283168
[17]	validation_0-error:0.274255
[18]	validation_0-error:0.267889
[19]	validation_0-error:0.268398
[20]	validation_0-error:0.270181
[21]	validation_0-error:0.266361
[22]	validation_0-error:0.265597
[23]	validation_0-error:0.266616
[24]	validation_0-error:0.268908
[25]	validation_0-error:0.265597
[26]	validation_0-error:0.26687
[27]	validation_0-error:0.263305
[28]	validation_0-erro

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=2, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [0]:
score = xgbmodel.predict_proba(test_x)  #预测分类概率
predict_y = xgbmodel.predict(test_x)
pp = [int(x[1]+0.5) for x in score]
get_score(test_y,score[:,1],pp)

acc:  0.7848230201171378 auc:  0.8622023499510426 f1_score:  0.7983774755428298 
precision:  0.7951520912547528 recall:  0.8016291327264016


/home/bil/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


# pra

In [0]:
with open('/home/bil/huangtao/project/data/competition/data_new/pra_dict.pickle', 'rb') as f:
    pra_dict = pickle.load(f)
with open('/home/bil/huangtao/project/data/competition/data_new/pra_negtive_dict.pickle', 'rb') as f:
    pra_negtive_dict = pickle.load(f)

In [0]:
#------------motif1---------------
pra_feature = []
flag = 0
for i in range(len(data_cut)):
    flag +=1
    bid1 = data_cut.loc[i,'bid1']
    bid2 = data_cut.loc[i,'bid2']
    pair = bid1 + '#' + bid2
    if pair in pra_dict:
        val = pra_dict[pair]
    else:
        val = pra_negtive_dict[pair]
    pra_feature += [val]
    if flag % 100 == 0:print '%d / %d\r' %(flag, len(data_cut)),

In [0]:
data_x = np.concatenate([ pra_feature],axis = 1)

In [0]:
data_x = np.array(data_x)
print data_x.shape
train_y = label[train_index]
test_y = label[test_index]
train_x = data_x[train_index,:]
test_x = data_x[test_index,:]

(39262, 100)


In [0]:
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
print '逻辑回归模型:\n',model

逻辑回归模型:
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


In [0]:
model.fit(train_x,train_y)
score = model.predict_proba(test_x)  #预测分类概率
predict_y = model.predict(test_x)
acc = model.score(test_x,test_y)
pp = [int(x[1]+0.5) for x in score]
get_score(test_y,score[:,1],pp)

acc:  0.7048637636872931 auc:  0.7702899418762109 f1_score:  0.6949197157146618 
precision:  0.7710280373831776 recall:  0.6324868231911835


In [0]:
xgbmodel.fit(train_x,train_y,early_stopping_rounds=10,eval_set=[(test_x,test_y)],eval_metric="error")

[0]	validation_0-error:0.32111
Will train until validation_0-error hasn't improved in 10 rounds.
[1]	validation_0-error:0.307614
[2]	validation_0-error:0.321365
[3]	validation_0-error:0.305067
[4]	validation_0-error:0.314489
[5]	validation_0-error:0.298956
[6]	validation_0-error:0.300993
[7]	validation_0-error:0.299211
[8]	validation_0-error:0.298701
[9]	validation_0-error:0.298192
[10]	validation_0-error:0.291571
[11]	validation_0-error:0.291317
[12]	validation_0-error:0.29208
[13]	validation_0-error:0.289789
[14]	validation_0-error:0.280112
[15]	validation_0-error:0.277311
[16]	validation_0-error:0.280112
[17]	validation_0-error:0.269417
[18]	validation_0-error:0.268144
[19]	validation_0-error:0.268398
[20]	validation_0-error:0.263051
[21]	validation_0-error:0.257703
[22]	validation_0-error:0.256175
[23]	validation_0-error:0.255157
[24]	validation_0-error:0.253374
[25]	validation_0-error:0.252101
[26]	validation_0-error:0.251337
[27]	validation_0-error:0.248026
[28]	validation_0-erro

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=2, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [0]:
score = xgbmodel.predict_proba(test_x)  #预测分类概率
predict_y = xgbmodel.predict(test_x)
pp = [int(x[1]+0.5) for x in score]
get_score(test_y,score[:,1],pp)

acc:  0.7855869620575503 auc:  0.86875182287869 f1_score:  0.7971084337349398 
precision:  0.8017450315075133 recall:  0.7925251557259224


/home/bil/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


# path

In [0]:
with open('/home/bil/huangtao/project/data/competition/data_new/path_dict.pickle', 'rb') as f:
    path_dict = pickle.load(f)

In [0]:
#------------motif1---------------
path_node = []#[poi*3,co_sq, dis_sq]*5
path_feature = []
flag = 0
for i in range(len(data_cut)):
    flag +=1
    bid1 = data_cut.loc[i,'bid1']
    bid2 = data_cut.loc[i,'bid2']
    pair = bid1 + '#' + bid2
    val = path_dict[pair]#[poi*3,co_sq, dis_sq]*5
    paths = val
    pn = []
    pf = []
    for path in paths:
        
        if path[0] != []:
            pn += [[poi_embed_dict[path[0][0]][0], poi_embed_dict[path[0][1]][0], poi_embed_dict[path[0][2]][0]]]
            feature = path[1] + path[2]
        else:
            pn += [[[0] * 211] * 3]
            feature = [0] * 6
        pf += [feature]
    path_node += [pn]
    path_feature += [pf]
    if flag % 100 == 0:print '%d / %d\r' %(flag, len(data_cut)),

In [0]:
path_node = np.array(path_node)
path_feature = np.array(path_feature)
print path_node.shape, path_feature.shape

(39262, 5, 3, 211) (39262, 5, 6)


In [0]:
print 'save...'
with open('/home/bil/huangtao/project/data/competition/data_new/gpu_motif/path_node.pickle','wb') as f:
    pickle.dump(path_node,f)
print 'save...'
with open('/home/bil/huangtao/project/data/competition/data_new/gpu_motif/path_feature.pickle','wb') as f:
    pickle.dump(path_feature,f)

save...
save...


In [0]:
with open('/home/bil/huangtao/project/data/competition/data_new/gpu_motif/path_node.pickle','rb') as f:
    path_node = pickle.load(f)
with open('/home/bil/huangtao/project/data/competition/data_new/gpu_motif/path_feature.pickle','rb') as f:
    path_feature = pickle.load(f)

save...
save...


In [0]:
print path_node.shape, path_feature.shape

(39262, 5, 3, 211) (39262, 5, 6)


In [0]:
path_f = path_feature[:,0,:]
path_n = path_node[:,0,:,:].reshape(-1,3*211)

In [0]:
data_x = np.concatenate([co_feature, bid1_embed, bid2_embed, path_f, region1],axis = 1)

In [0]:
import xgboost as xgb
xgbmodel = xgb.XGBClassifier(n_estimators=100, 
                             silent=True, 
                             objective='binary:logistic',
                             max_depth=2, 
                             learning_rate=0.1)
xgbmodel.fit(train_x,train_y,early_stopping_rounds=10,eval_set=[(test_x,test_y)],eval_metric="error")

[0]	validation_0-error:0.32111
Will train until validation_0-error hasn't improved in 10 rounds.
[1]	validation_0-error:0.307614
[2]	validation_0-error:0.321365
[3]	validation_0-error:0.305067
[4]	validation_0-error:0.314489
[5]	validation_0-error:0.298956
[6]	validation_0-error:0.300993
[7]	validation_0-error:0.299211
[8]	validation_0-error:0.298701
[9]	validation_0-error:0.298192
[10]	validation_0-error:0.291571
[11]	validation_0-error:0.291317
[12]	validation_0-error:0.29208
[13]	validation_0-error:0.289789
[14]	validation_0-error:0.280112
[15]	validation_0-error:0.277311
[16]	validation_0-error:0.280112
[17]	validation_0-error:0.269417
[18]	validation_0-error:0.268144
[19]	validation_0-error:0.268398
[20]	validation_0-error:0.263051
[21]	validation_0-error:0.257703
[22]	validation_0-error:0.256175
[23]	validation_0-error:0.255157
[24]	validation_0-error:0.253374
[25]	validation_0-error:0.252101
[26]	validation_0-error:0.251337
[27]	validation_0-error:0.248026
[28]	validation_0-erro

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=2, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [0]:
score = xgbmodel.predict_proba(test_x)  #预测分类概率
predict_y = xgbmodel.predict(test_x)
pp = [int(x[1]+0.5) for x in score]
get_score(test_y,score[:,1],pp)

acc:  0.7855869620575503 auc:  0.86875182287869 f1_score:  0.7971084337349398 
precision:  0.8017450315075133 recall:  0.7925251557259224


/home/bil/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
